In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

# Numerical Modeling Assignment 2.1
### Ilaova Alina, BS4-DS2

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

We need to find $S_{3}(x)$ such that:
* $S_3(x_i) = f(x_i)$ for $i \in [0, N-1]$
* $S_{3}(x) = \bigcup \limits_{i=0}^{N-1} S_{3, i}(x) \in C_2[a, b]$
* $S_{3, i}(x) = a_{0, i} + a_{1, i}(x - x_i) + a_{2, i}(x - x_i)^2 + a_{3, i}(x - x_i)^3$

### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

Compatibility conditions:
* **functional:**

$\delta S_{3, i}(x)|_{x=x_i} = S_{3, i}(x_i) - f(x_i) = 0$

$\delta S_{3, i}(x)|_{x=x_{i+1}} = S_{3, i}(x_{i+1}) - f(x_{i+1}) = 0$

* **differential:**

$\delta S_{3, i}^{''}(x)|_{x=x_i} = S_{3, i}^{''}(x_i) - f^{''}(x_i) = 0$

$\delta S_{3, i}^{''}(x)|_{x=x_{i+1}} = S_{3, i}^{''}(x_{i+1}) - f^{''}(x_{i+1}) = 0$

### 3. Formulate stitching conditions [mathematical formula]

Stitching condition:

$S_{3, i-1}'(x)|_{x=x_i} = S_{3, i}'(x)|_{x=x_i}$

### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

Stitching condition guarantees us that the function will be continous, differential compatibility conditions guarantee the smoothnes of spline function.

But these conditions don't guarantee us that the whole spline function will be two times differentiable. As we know, each spline is represented as cubic polynomial, which is infinitely differentiable no matter what its coefficients are. And because of that we can say for sure that the whole spline is two times differentiable.

### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

We need to find polynoms: 

$S_{3, i}(x) = a_{0, i} + a_{1, i}(x - x_i) + a_{2, i}(x - x_i)^2 + a_{3, i}(x - x_i)^3$

$S_{3, i}^{''}(x) = 2a_{2,i} + 6a_{3,i}x - 6a_{3,i}x_i$

---

And substituting this polynom into compatibility conditions, we get:

(1) $a_{0,i} + a_{1,i}(x_i-x_i) + a_{2,i}(x_i-x_i)^2 + a_{3,i}(x_i-x_i)^3 - f(x_i) = 0$

(2) $a_{0,i} + a_{1,i}(x_{i+1}-x_{i}) + a_{2,i}(x_{i+1}-x_{i})^2 + a_{3,i}(x_{i+1}-x_{i})^3 - f(x_{i+1}) = 0$

(3) $2a_{2,i} + 6a_{3,i}x_i - 6a_{3,i}x_i - f^{''}(x_i) = 0$

(4) $2a_{2,i} + 6a_{3,i}x_{i+1} - 6a_{3,i}x_{i} - f^{''}(x_{i+1}) = 0$

or:

(1) $a_{0,i} - f(x_i) = 0$

(2) $a_{0,i} + a_{1,i}(x_{i+1}-x_{i}) + a_{2,i}(x_{i+1}-x_{i})^2 + a_{3,i}(x_{i+1}-x_{i})^3 - f(x_{i+1}) = 0$

(3) $2a_{2,i} - f^{''}(x_i) = 0$

(4) $2a_{2,i} + 6a_{3,i}x_{i+1} - 6a_{3,i}x_{i} - f^{''}(x_{i+1}) = 0$

---

$f^{''}(x_i) = m_i, \Delta m_i = m_{i+1} - m_i,$

$f(x_i) = f_i, \Delta f_i = f_{i+1} - f_i$

$h_{i+1} = x_{i+1} - x_i$

---

$a_{0,i} = f_i$

---

$a_{2,i} = \frac{m_i}{2}$

---

**Substituting $a_{2,i}$ in the (4) gives us:**

$m_i + 6a_{3,i}x_{i+1} - 6a_{3,i}x_{i} - m_{i+1} = 0$

$6a_{3,i}(x_{i+1} - x_{i}) = m_{i+1} - m_i$

$6h_{i+1}a_{3,i} = \Delta m_i$

$a_{3,i} = \frac{\Delta m_i}{6h_{i+1}}$

---

$f_i + a_{1,i}(x_{i+1}-x_{i}) + \frac{m_i}{2}(x_{i+1}-x_{i})^2 + \frac{\Delta m_i}{6h_{i+1}}(x_{i+1}-x_{i})^3 - f_{i+1} = 0$

$a_{1,i}h_{i+1} = f_{i+1} - f_{i} - \frac{m_i}{2}h_{i+1}^2 - \frac{\Delta m_i}{6h_{i+1}}h_{i+1}^3$

$a_{1,i} = \frac{1}{h_{i+1}}\Delta f_i - \frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}\Delta m_i$

---

$S_{3, i}(x) = f_i + (\frac{1}{h_{i+1}}\Delta f_i - \frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}\Delta m_i)(x-x_i) + \frac{m_i}{2}(x-x_i)^2 + \frac{1}{6h_{i+1}}\Delta m_i(x-x_i)^3$

---

$S_{3, i}^{'}(x_i) = S_{3, i-1}^{'}(x_i)$

$\frac{1}{h_{i+1}}\Delta f_i - \frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}\Delta m_i + m_i(x_i-x_i) + \frac{1}{2h_{i+1}}\Delta m_i (x_i-x_i)^2 = \frac{1}{h_i}\Delta f_{i-1} - \frac{h_i}{2}m_{i-1} - \frac{h_i}{6}\Delta m_{i-1} + m_{i-1}(x_i-x_{i-1}) + \frac{1}{2h_i}\Delta m_{i-1}(x_i-x_{i-1})^2$

$\frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_i} = \frac{h_{i+1}}{2}m_i + \frac{h_{i+1}}{6}\Delta m_i  - \frac{h_i}{2}m_{i-1} - \frac{h_i}{6}\Delta m_{i-1} + m_{i-1}h_i + \frac{h_i}{2}\Delta m_{i-1}$

$\frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_i} = \frac{h_{i+1}}{2}m_i + \frac{h_{i+1}}{6}(m_{i+1}-m_i) - \frac{h_i}{2}m_{i-1} - \frac{h_i}{6}(m_{i}-m_{i-1}) + m_{i-1}h_i + \frac{h_i}{2}(m_{i}-m_{i-1})$

$\frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_i} = m_{i-1}(-\frac{h_i}{2} + \frac{h_i}{6} + h_i - \frac{h_i}{2}) + m_i(\frac{h_{i+1}}{2} - \frac{h_{i+1}}{6} - \frac{h_i}{6} + \frac{h_i}{2}) + \frac{h_{i+1}}{6}m_{i+1}$

---

$\frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_i} = \frac{h_i}{6}m_{i-1} + \frac{h_i + h_{i+1}}{3}m_i + \frac{h_{i+1}}{6}m_{i+1}$

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

Matrix representation:
$$
\begin{bmatrix}
    \frac{h_1+h_2}{3} & \frac{h_2}{6}     & 0                 & \dots & 0     & 0 
    & \frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\
    \frac{h_2}{6}     & \frac{h_2+h_3}{3} & \frac{h_3}{6}     & \dots & 0     & 0   
    & \frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2}\\
    0                 & \frac{h_3}{6}     & \frac{h_3+h_4}{3} & \dots & 0     & 0    
    & \frac{\Delta f_3}{h_4} - \frac{\Delta f_2}{h_3} \\
    \dots             & \dots             & \dots             & \dots & \dots & \dots & \dots \\
    0                 & 0                 & 0                 & \dots & \frac{h_{n-1}}{6} & \frac{h_{n-1}+h_n}{3}
    & \frac{\Delta f_{n-1}}{h_n} - \frac{\Delta f_{n-2}}{h_{n-1}}
\end{bmatrix}$$

### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

This system is not closed with respect to $m_i$. We need equations for $m_0$ and $m_n$. There are several methods for that, for example, we can choose them to be:
$m_0 = m_n = 0$.

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

We will demonstrate Gauss elimination on the first two rows of the matrix:

**Step 1**: divide 1st row by $\frac{h_1+h_2}{3}$

Result (1st row):

$
\begin{bmatrix}
    1 & \frac{h_2}{2(h_1+h_2)} & 0 & \dots & 0 & (\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1})\frac{3}{h_1+h_2}
\end{bmatrix}
$

**Step 2**: subtract 1st row multiplied by $\frac{h_2}{6}$ from 2nd row:

Result (1st and 2nd rows):

$
\begin{bmatrix}
    1 & \frac{h_2}{2(h_1+h_2)} & 0 & \dots & 0 & (\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1})\frac{3}{h_1+h_2}\\
    0 & \frac{-h_2^2 + 4(h_1+h_2)(h_2+h_3)}{12(h_1+h_2)} & \frac{h_3}{6} & \dots & 0 & \frac{2(\Delta f_2 h_2 - \Delta f_1 h_3)h_1(h_1+h_2) - (\Delta f_1 h_1 - \Delta f_0 h_2)h_2 h_3}{2 h_1 h_2 h_3 (h_1 + h_2)}
\end{bmatrix}
$

**Step 3**: divide 2nd row by $\frac{-h_2^2 + 4(h_1+h_2)(h_2+h_3)}{12(h_1+h_2)}$:

Result (1st and 2nd rows):

$
\begin{bmatrix}
    1 & \frac{h_2}{2(h_1+h_2)} & 0 & \dots & 0 & (\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1})\frac{3}{h_1+h_2}\\
    0 & 1 & \frac{2h_3(h_1+h_2)}{4(h_2+h_3)(h_1+h_2)-h_2} & \dots & 0 & \frac{6 h_2^2 h_3 \Delta f_0 + 12(h_1+h_2) \Delta f_2 + h_2 h_1 - 12(h_1+h_2)\Delta f_1 h_1 h_3 - h_1 h_2 h_3 6 \Delta f_1}{h_1 h_2 h_3 (4(h_2+h_3)(h_1 + h_2) - h_2^2)}
\end{bmatrix}
$

And after performing these operations on other rows, we get following structure of matrix:

$$
\begin{bmatrix}
    1     & -P_1  & 0     & 0     & \dots & Q_1   \\
    0     & 1     & -P_2  & 0     & \dots & Q_2   \\
    0     & 0     & 1     & -P_3  & \dots & Q_3   \\
    \dots & \dots & \dots & \dots & \dots & \dots \\
    0     & 0     & 0     & 1     & \dots & Q_n   \\
\end{bmatrix}$$

where $P_i$ and $Q_i$ are computed using formaulas from question 9.

### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

Forward pass:

$P_i = \frac{h_{i+1}}{- 2(h_{i+1} + h_i) - h_{i}P_{i-1}}, P_1 = -\frac{h_2}{2(h_2+h_1)}$

$Q_i = \frac{\frac{h_i}{6}Q_{i-1} - \frac{\Delta f_i}{h_{i+1}} + \frac{\Delta f_{i-1}}{h_{i}}}{\frac{h_{i+1} + h_i}{3} - \frac{h_i}{6}P_{i-1}}, Q_1 = \frac{3(\frac{\Delta f_1}{h_{2}} - \frac{\Delta f_{0}}{h_{1}})}{h_{2} + h_1}$

Backward pass:

$x_n = \frac{\frac{h_n}{6}Q_{n-1} - \frac{\Delta f_n}{h_{n+1}} + \frac{\Delta f_{n-1}}{h_{n}}}{\frac{h_{n+1} + h_n}{3} - \frac{h_n}{6}P_{n-1}}$

$x_i = \frac{h_{i+1}}{- 2(h_{i+1} + h_i) - h_{i}P_{i-1}} x_{i+1} + \frac{\frac{h_i}{6}Q_{i-1} - \frac{\Delta f_i}{h_{i+1}} + \frac{\Delta f_{i-1}}{h_{i}}}{\frac{h_{i+1} + h_i}{3} - \frac{h_i}{6}P_{i-1}}$

### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [1]:
def forward_pass(alpha, beta, gamma, delta):
    pass

def backward_pass(alpha, beta, delta, p, q):
    pass

### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

Cubic spline error:

$max|f^p - S_3^p| \leqslant \mu_4 h^{4-p}$

but this estimation of the error is valid only if the function $f$ is 4 times defferentiable and this function is unknown to us so we can't assume anything about its form and in the case if $f \notin C^4$ we can't say anything about the method error.

### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

Forward pass - $O(n)$, backward pass - $O(n)$ -> $O(2n) = O(n)$

### Congrats!